# Ormir-mids: dcm2omids 

- By: [Simone Poncioni](https://github.com/simoneponcioni), [Serena Bonaretti](https://sbonaretti.github.io/)
- Notebook created by merging and modifying a notebook by Leonardo Barzaghi, [Donnie Cameron](https://github.com/DC-3T), Judith Cueto Fernandez, Jilmen Quintiens, [Francesco Santini](https://github.com/fsantini), and a notebook by [Gianluca Iori](https://github.com/gianthk) and Francesco Chiumento
- Code license: Apache 2.0
- Narrative license: CC-BY-NC-SA
- How to cite: Sarah Manske, Mahdi Hosseinitabatabaei, Pholpat Durongbhan, Michael Kuczynski, Simone Poncioni, Gianluca Iori, Serena Bonaretti. *Why and how to share MSK imaging data?* Worskhop at the 24th International Workshop on Quantitative Musculoskeletal Imaging (QMSKI). The Barossa Valley, South Australia. November 3-8, 2024.

---

### Aims

The [ORMIR-MIDS package](https://github.com/ormir-mids/ormir-mids/tree/Jupyter) converts musculoskeletal DICOM images into a data structure inspired by [BIDS](https://bids.neuroimaging.io/)[<sup id="fn1-back">1</sup>](#fn1), following the [ORMIR-MIDS specs](https://ormir-mids.github.io/specs.html)

In this notebook you will learn how to:

1. Convert images to ORMIR-MIDS data structure 
2. Explore an ORMIR-MIDS data structure
3. Explore an image volume (before conversion)

---

### Installations

- Let's install ORMIR-MIDS:

In [1]:
# %pip install ormir-mids

# Alternatively, you can install the package from the source code

!git clone https://github.com/ormir-mids/ormir-mids.git
%cd ormir-mids
%pip install -e .

Cloning into 'ormir-mids'...
remote: Enumerating objects: 1289, done.
remote: Counting objects: 100% (555/555), done.
remote: Compressing objects: 100% (297/297), done.
remote: Total 1289 (delta 308), reused 489 (delta 253), pack-reused 734 (from 1)
Receiving objects: 100% (1289/1289), 46.14 MiB | 24.54 MiB/s, done.
Resolving deltas: 100% (502/502), done.
/home/simoneponcioni/Documents/02_PROJECTS/ormir-mids/jupyter/ormir-mids
Obtaining file:///home/simoneponcioni/Documents/02_PROJECTS/ormir-mids/jupyter/ormir-mids
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ormir-mids (pyproject.toml) ... done
  Created wheel for ormir-mids: filename=ormir_mids-0.0.6-0.editable-py3-none-any.whl size=7668 sha256=1c0a07d46cbd31f830155cc52573cfbf65361342d3d9f00e43c303e30cb7d00f
  Stored in directory: /tmp/pip-ephem-whee

- To explore the folder and data structure, we will need the [*Directory Tree*](`https://pypi.org/project/directory-tree/`) package:

In [ ]:
# to explore the folder and data structure
%pip install directory_tree
# to visualize images in a jupyter notebook
%pip install k3d
# to convert to SimpleITK
%pip install SimpleITK
# print out our machine characteristics and the version of the packages (for reproducibility)
%pip install watermark

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Or if you use conda:
> `conda install -c conda-forge directory-tree k3d watermark`

> `conda install -c https://conda.anaconda.org/simpleitk SimpleITK`

### Imports 

- We will need to use the following packages:

In [1]:
import ormir_mids
from ormir_mids.dcm2omids import convert_dicom_to_ormirmids
from ormir_mids.utils.io import find_omids, load_omids, load_dicom

ModuleNotFoundError: No module named 'ormir_mids'

In [2]:
import zipfile
from pathlib import Path

import requests
import SimpleITK as sitk
import k3d
import numpy as np

from directory_tree import DisplayTree

### Functions

- This is a function to download and unzip the folder containing the image data:

In [3]:
def download_and_unzip(url, extract_to='.'):
    """
    Downloads a ZIP file from the specified URL and extracts it.

    Parameters
    ----------
    url : str
        The URL of the ZIP file to download.
    extract_to : str, optional
        The directory to extract the contents to. Defaults to the current directory.

    Returns
    -------
    None
    """
    
    extract_to_path = Path(extract_to)
    if not extract_to_path.exists():
        extract_to_path.mkdir(parents=True)
    local_zip_path = extract_to_path / 'downloaded.zip'

    response = requests.get(url)
    response.raise_for_status()  # Ensure we notice bad responses

    with local_zip_path.open('wb') as file:
        file.write(response.content)

    with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_path)

    local_zip_path.unlink()

### Variables

- Here are the variables we will use:

In [4]:
# zipped data folder to download
zipped_folder = "https://github.com/ormir-mids/ormir-mids/raw/main/dicom.zip"
# designated unzipped data folder
dicom_folder = "data"
# folder that will contain the images in ORMIR-MIDS data structure
omids_folder = "omids_data"

---

## Getting the data

- Extract the data from the provided zipped folder and save them in *dicom_folder*:

In [5]:
download_and_unzip(zipped_folder, extract_to=dicom_folder)

- Let's have a look at what data are available. We'll use the Python package `directory_tree`, which allows us to display our directory structure:

In [6]:
DisplayTree(dirPath=dicom_folder,
            stringRep=False,
            header=True,
            maxDepth=3,
            showHidden=False,
            sortBy=2, # 0 - Default, 1 - Files First, 2 - Directories First
            )


$ Operating System : Linux
$ Path : data

*************** Directory Tree ***************

data/
└── dicom/
    ├── GE_MEGRE_B0/
    │   ├── IM0
    │   ├── IM1
    │   ├── IM10
    │   ├── IM100
    │   ├── IM101
    │   ├── IM102
    │   ├── IM103
    │   ├── IM104
    │   ├── IM105
    │   ├── IM106
    │   ├── IM107
    │   ├── IM108
    │   ├── IM109
    │   ├── IM11
    │   ├── IM110
    │   ├── IM111
    │   ├── IM112
    │   ├── IM113
    │   ├── IM114
    │   ├── IM115
    │   ├── IM116
    │   ├── IM117
    │   ├── IM118
    │   ├── IM119
    │   ├── IM12
    │   ├── IM120
    │   ├── IM121
    │   ├── IM122
    │   ├── IM123
    │   ├── IM124
    │   ├── IM125
    │   ├── IM126
    │   ├── IM127
    │   ├── IM128
    │   ├── IM129
    │   ├── IM13
    │   ├── IM130
    │   ├── IM131
    │   ├── IM132
    │   ├── IM133
    │   ├── IM134
    │   ├── IM135
    │   ├── IM136
    │   ├── IM137
    │   ├── IM138
    │   ├── IM139
    │   ├── IM14
    │   ├── IM140
    │   ├── IM14

- As you can see, these are one Multi-Echo GRadient-Echo (MEGRE) image from GE and one Multi-Echo Spin-Echo (MESE) image from Philips
- The GE data comprise multiple DICOM files, whereas the Philips data are contained in a single, large file
- You can also browse the files inside their dicom directory from the file explorer

---
# 1. Converting to ORMIR-MIDS data structure

- Let's convert the images from out-of-scanner `.dcm` to the ORMIR-MIDS structure:

In [7]:
convert_dicom_to_ormirmids(input_folder=dicom_folder+"/dicom/", output_folder=omids_folder, anonymize='anon', recursive=True)

data/dicom/GE_MEGRE_B0
Volume compatible with MESE_Philips_Magnitude
Volume saved
Volume compatible with MESE_Philips_Phase
Volume saved
Volume compatible with MESE_Philips_ReconstructedT2
Volume saved
Volume compatible with MEGRE_Philips_Reconstructed
Volume saved
Volume compatible with MEGRE_GE_Magnitude
Volume saved
Volume compatible with MEGRE_GE_Phase
Volume saved
Volume compatible with MEGRE_GE_Real
Volume saved
Volume compatible with MEGRE_GE_Imaginary
Volume saved


- The output data are saved to the new directory called `omids_folder`. Let's see explore it:

In [8]:
DisplayTree(dirPath=omids_folder,
            stringRep=False,
            header=True,
            maxDepth=3,
            showHidden=False,
            sortBy=2, # 0 - Default, 1 - Files First, 2 - Directories First
            )


$ Operating System : Linux
$ Path : omids_data

*************** Directory Tree ***************

omids_data/
├── mr-anat/
│   ├── anon_megre.json
│   ├── anon_megre.nii.gz
│   ├── anon_megre_extra.json
│   ├── anon_megre_imag.json
│   ├── anon_megre_imag.nii.gz
│   ├── anon_megre_imag_extra.json
│   ├── anon_megre_imag_patient.json
│   ├── anon_megre_patient.json
│   ├── anon_megre_ph.json
│   ├── anon_megre_ph.nii.gz
│   ├── anon_megre_ph_extra.json
│   ├── anon_megre_ph_patient.json
│   ├── anon_megre_real.json
│   ├── anon_megre_real.nii.gz
│   ├── anon_megre_real_extra.json
│   ├── anon_megre_real_patient.json
│   ├── anon_mese.json
│   ├── anon_mese.nii.gz
│   ├── anon_mese_extra.json
│   ├── anon_mese_patient.json
│   ├── anon_mese_ph.json
│   ├── anon_mese_ph.nii.gz
│   ├── anon_mese_ph_extra.json
│   └── anon_mese_ph_patient.json
└── mr-quant/
    ├── anon_megre_reco.json
    ├── anon_megre_reco.nii.gz
    ├── anon_megre_reco_extra.json
    ├── anon_megre_reco_patient.json
    

- The converted data are sorted under `mr-anat` for anatomical images and `mr-quant` for quantitative maps
- Every dataset is composed of:
  -  A `.nii.gz` file containing the image data
  -  A set of `.json` files containing the headers. Specifically:
     - A simple `.json` file containing useful information about the data
     - A `_patient.json` file containing private patient data. *Delete this file if you want to anonymize your data!*
     - A `_extra.json` file containing extra information that can be used to reconstruct the DICOM dataset from the ORMIR-MIDS data, so it can be stored again in PACS



---
# 2. Exploring the ORMIR-MIDS data structure

### 2.1 Finding images
- Let's find the `mese` image in the folder:

In [9]:
mese_data_list = find_omids(omids_folder, 'mese')
print(mese_data_list)

['omids_data/mr-anat/anon_mese.nii.gz']


- *Exercise: Find the `mese_ph` image in the folder:*

### 2.2 Reading an image

In [10]:
omids_mese = load_omids(mese_data_list[0])

### 2.3 Reading `.json` files

- Each image volume has three dictionaries (corresponding to the three `.json` files) associated with it:
  - `omids_header`containing relevant information
  - `patient_header` containing privacy-relevant information (file is missing in case of anonymization)
  - `extra_header`: a collection of the remaining DICOM tags

- Let's read the patient header:

In [11]:
omids_mese.patient_header

{'PatientName': {'Alphabetic': 'Npmr0_0029^^^^'},
 'PatientID': '7732486',
 'Birthdate': '19900101',
 'Age': '031Y',
 'InstitutionName': 'LUMC',
 'InstitutionAddress': 'Albinusdreef 2',
 'InstitutionalDepartmentName': 'Radiologie',
 'ReferringPhysician': {'Alphabetic': 'Radiologie^^^^'},
 'AccessionNumber': '0000000008004743'}

- *Exercise: Read the extra header*:

In [12]:
omids_mese.extra_header

{'00080005': {'vr': 'CS', 'Value': ['ISO_IR 100']},
 '00080008': {'vr': 'CS', 'Value': ''},
 '00080012': {'vr': 'DA', 'Value': ['20210927']},
 '00080013': {'vr': 'TM', 'Value': ['123342.307']},
 '00080014': {'vr': 'UI', 'Value': ['1.3.46.670589.11.89.5']},
 '00080016': {'vr': 'UI', 'Value': ['1.2.840.10008.5.1.4.1.1.4']},
 '00080018': {'vr': 'UI',
  'Value': [[['1.3.46.670589.11.71290.5.0.8408.2021092713141147611'],
    ['1.3.46.670589.11.71290.5.0.8408.2021092713141147612'],
    ['1.3.46.670589.11.71290.5.0.8408.2021092713141149613'],
    ['1.3.46.670589.11.71290.5.0.8408.2021092713141151614'],
    ['1.3.46.670589.11.71290.5.0.8408.2021092713141151615'],
    ['1.3.46.670589.11.71290.5.0.8408.2021092713141152616'],
    ['1.3.46.670589.11.71290.5.0.8408.2021092713141154617'],
    ['1.3.46.670589.11.71290.5.0.8408.2021092713141154618'],
    ['1.3.46.670589.11.71290.5.0.8408.2021092713141155619'],
    ['1.3.46.670589.11.71290.5.0.8408.2021092713141157620'],
    ['1.3.46.670589.11.71290.5.

---
## 3. Exploring `.dcm` images before conversion

### 3.1 Load DICOM image, header, and metadata

- ORMIR-MIDS also allows us to explore the content of the `.dcm`, before being converted to the ORMIR-MIDS structure:

- Load the MESE image in `.dcm`:

In [13]:
img_dcm = load_dicom('data/dicom/Philips_MESE_T2.dcm')

- Explore the `.dcm` image header:

In [14]:
print(f'Scanner Manufacturer: {img_dcm.bids_header["Manufacturer"]}')
print(f'Scanner Orientation: {img_dcm.orientation}')
print(f'Image Type: {img_dcm.dtype}')
print(f'Image Shape: {img_dcm.shape}')
print(f'Scanner Origin: {img_dcm.scanner_origin}')
print(f'Scanner spacing: {img_dcm.pixel_spacing}')

Scanner Manufacturer: Philips Medical Systems
Scanner Orientation: ('AP', 'RL', 'IS')
Image Type: uint16
Image Shape: (176, 176, 210)
Scanner Origin: (240.9552, 232.94, -1053.7351)
Scanner spacing: (2.84090900421142, 2.84090900421142, 10.0)


- Crop the volume. To create a separate subvolume we can do the following. Metadata will be sliced appropriately.

In [15]:
img_dcm_subvolume = img_dcm[50:90, 50:90, 30:70]
print(f'Original Shape: {img_dcm.shape}')
print(f'Subvolume Shape: {img_dcm_subvolume.shape}')

Original Shape: (176, 176, 210)
Subvolume Shape: (40, 40, 40)


### 3.2 Convert to SimpleITK

- Since many of us use SimpleITK in our codes, here is a conversion example:

In [16]:
img_sitk = img_dcm.to_sitk()
print(f'Before conversion, img_dcm is of type: {type(img_dcm)}\nAfter conversion, img_sitk is of type: {type(img_sitk)}')

Before conversion, img_dcm is of type: <class 'voxel.med_volume.MedicalVolume'>
After conversion, img_sitk is of type: <class 'SimpleITK.SimpleITK.Image'>


- Inspect SimpleITK image contents:

In [17]:
print(img_sitk)

Image (0x55f6002e7ae0)
  RTTI typeinfo:   itk::Image<unsigned short, 3u>
  Reference Count: 1
  Modified Time: 1671
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 0
  UpdateMTime: 0
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [176, 176, 210]
  BufferedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [176, 176, 210]
  RequestedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [176, 176, 210]
  Spacing: [2.84091, 2.84091, 10]
  Origin: [-240.955, -232.94, -1053.74]
  Direction: 
-0 1 0
1 -0 0
0 0 1

  IndexToPointMatrix: 
0 2.84091 0
2.84091 0 0
0 0 10

  PointToIndexMatrix: 
0 0.352 0
0.352 0 0
0 0 0.1

  Inverse Direction: 
0 1 0
1 0 0
0 0 1

  PixelContainer: 
    ImportImageContainer (0x55f5f74272d0)
      RTTI typeinfo:   itk::ImportImageContainer<unsigned long, unsigned shor

### 3.3 Visualize with k3d

In [18]:
# Extract spacing, origin, and dimensions from a vtkImageData object. These are important properties that describe the resolution, position, and physical size of the image.
spacing = img_sitk.GetSpacing()
origin = img_sitk.GetOrigin()
dimensions = img_sitk.GetSize()

print(spacing)
print(origin)
print(dimensions)

transform_matrix = np.array([
    [spacing[0] * dimensions[0], 0, 0, origin[0]],
    [0, spacing[1] * dimensions[1], 0, origin[1]],
    [0, 0, spacing[2] * dimensions[2], origin[2]],
    [0, 0, 0, 1]
])

def image2np(image):
    imnp = sitk.GetArrayFromImage(image)
    # Transpose and flip the array to have the same indexing and direction as the original image
    imnp = np.transpose(imnp, (2, 1, 0))
    imnp = np.flip(imnp, axis=2)
    # Enforce float16 type (best for k3d)
    imnp = imnp.astype(np.float16)
    return imnp

imnp = image2np(img_sitk)
vmin, vmax = np.percentile(imnp, [0.01, 99.99])

transform_s = k3d.transform(custom_matrix=transform_matrix)

# Instantiate the volume plot using k3d-jupyter
plt_volume = k3d.volume(imnp,
                        transform=transform_s,
                        color_map=k3d.colormaps.basic_color_maps.Binary,
                        samples=256,
                        alpha_coef=150,
                        color_range=[vmin, vmax])
plot = k3d.plot()
plot += plt_volume

# Display the plot inline
plot.display()

(2.84090900421142, 2.84090900421142, 10.0)
(-240.9552, -232.94, -1053.7351)
(176, 176, 210)


Output()

---
## Dependencies

- For future reproducibility of the workflow:

In [19]:
%load_ext watermark

%watermark
%watermark --iversions

Last updated: 2024-10-30T11:02:34.277869+01:00

Python implementation: CPython
Python version       : 3.7.12
IPython version      : 7.34.0

Compiler    : GCC 9.4.0
OS          : Linux
Release     : 6.8.0-47-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 16
Architecture: 64bit

SimpleITK : 2.2.1
numpy     : 1.21.6
k3d       : 2.16.1
requests  : 2.31.0
ormir_mids: 0.0.6



---
<a name="ref"></a>
## References

[<sup id="fn1">1</sup>](#fn1-back) Gorgolewski, K., Auer, T., Calhoun, V. et al. The brain imaging data structure, a format for organizing and describing outputs of neuroimaging experiments. Sci Data 3, 160044 (2016). https://doi.org/10.1038/sdata.2016.44

---
## Acknowledgements
ORMIR MIDS is under development by memberes of the [ORMIR](https://www.ormir.org/)
community. 
The development started during the 2nd ORMIR workshop [Sharing and Curating Open Data in Musculoskeletal Imaging Research](https://github.com/ORMIRcommunity/2024_2nd_ORMIR_WS/blob/main/README.md), Zurich, Switzerland. 15-18 January 2024. 
ORMIR MIDS is based on 
[muscle-bids](https://github.com/muscle-bids/muscle-bids), developed during the 1st ORMIR workshop [Building the Jupyter Community in MSK Imaging Research - A Jupyter Commuity Workshop](https://github.com/JCMSK/2022_JCW/blob/main/README.md), Maastricht, The Netherlands, 9-11 June 2022; 
and [ORMIR-PyVoxel](https://github.com/ormir-mids/ormir-pyvoxel) a tool to handle medical image I/O modified from [pyVoxel](https://github.com/pyvoxel/pyvoxel) by Arjun Desai.  


---
<a name="attribution"></a>

Notebook created using the [template](https://github.com/ORMIRcommunity/templates/blob/main/ORMIR_nb_template.ipynb) of the [ORMIR community](https://ormircommunity.github.io/) (version 1.0, 2023)